<a href="https://colab.research.google.com/github/ravindu16/-events-app-api-server/blob/main/Day3_LangGraph_agenticAI_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>